In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
def load_parquet_optional(p: Path):
    if p.exists():
        try:
            return pd.read_parquet(p)
        except Exception as e:
            print(f"Failed to read {p}: {e}")
            return None
    return None


def sanitize_numeric_series(s):
    return pd.to_numeric(s, errors='coerce')


# small plotting helper
def save_plot(fig, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(path, bbox_inches='tight')
    plt.close(fig)

In [3]:
# when pasted into a notebook, set SEASON variable manually
SEASON = 2020

PROJECT_ROOT = PROJECT_ROOT = Path.cwd().parents[1]
BASE_DIR = PROJECT_ROOT / "data" / "seasons" / str(SEASON)

SAVE_REPORT = False
OUT_DIR = BASE_DIR / 'eda_report'

OUT_DIR.mkdir(parents=True, exist_ok=True)
PLOTS_DIR = OUT_DIR / 'plots'
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Season folder: {BASE_DIR.resolve()}")

# %%
# Load normalized tables
matches_fp = BASE_DIR / 'matches.parquet'
players_fp = BASE_DIR / 'player_stats.parquet'
events_fp = BASE_DIR / 'events.parquet'
teams_fp = BASE_DIR / 'teams.parquet'

matches = load_parquet_optional(matches_fp)
players_stats = load_parquet_optional(players_fp)
events = load_parquet_optional(events_fp)
teams = load_parquet_optional(teams_fp)

print('Loaded:')
print(' matches:', getattr(matches, 'shape', None))
print(' players:', getattr(players_stats, 'shape', None))
print(' events :', getattr(events, 'shape', None))
print(' teams :', getattr(teams, 'shape', None))

Season folder: /home/kamil/projects/tipster/data/seasons/2020
Loaded:
 matches: (380, 17)
 players: (14643, 44)
 events : (11132, 9)
 teams : (20, 2)


In [4]:
def process_player_stats(player_stats: pd.DataFrame, drop_old_features=True) -> pd.DataFrame:
    df = player_stats.copy()

    # Ensure unified datetime for ordering
    if 'fixture_dt' not in df.columns and 'fixture_date' in df.columns:
        df['fixture_dt'] = pd.to_datetime(df['fixture_date'], errors='coerce')
    else:
        df['fixture_dt'] = pd.to_datetime(df.get('fixture_dt'), utc=True, errors='coerce')

    # Rating source if only games_rating exists
    if 'rating' not in df.columns and 'games_rating' in df.columns:
        df['rating'] = pd.to_numeric(df['games_rating'], errors='coerce')

    # Per-90 inputs
    df['minutes'] = pd.to_numeric(df.get('minutes'), errors='coerce').fillna(0)
    for col in ['shots_on','shots_total','passes_key']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
            df[f'{col}_per90'] = np.where(df['minutes'] > 0, (df[col]/df['minutes']) * 90.0, 0.0)

    df['minutes_share'] = np.where(df['minutes'] > 0, df['minutes'] / 90.0, 0.0)

    # Order → shift → EMA
    df = df.sort_values(['player_id','fixture_dt']).copy()
    base_cols = [c for c in ['rating','shots_on_per90','shots_total_per90','passes_key_per90','minutes_share'] if c in df.columns]

    for c in base_cols:
        df[f'{c}_shifted'] = df.groupby('player_id', group_keys=False)[c].shift(1)

    for c in base_cols:
        df[f'{c}_ema5'] = (
            df.groupby('player_id', group_keys=False)[f'{c}_shifted']
              .apply(lambda s: s.ewm(span=5, adjust=False).mean())
        )  # EMA per pandas docs [4][5]

    df.drop(columns=[f'{c}_shifted' for c in base_cols], inplace=True, errors='ignore')

    if drop_old_features:
        keep = [
            'fixture_id','player_id','team_id','fixture_dt',
            'rating_ema5','shots_on_per90_ema5','shots_total_per90_ema5',
            'passes_key_per90_ema5','minutes_share_ema5'
        ]
        df = df[[c for c in keep if c in df.columns]].copy()

    ema_cols = [c for c in df.columns if c.endswith('_ema5')]
    if ema_cols:
        df[ema_cols] = df[ema_cols].fillna(0)

    return df

# Example Usage
# Assuming player_stats is your DataFrame from parquet
# processed_df = process_player_stats(player_stats, drop_old_features=True)
# processed_df.to_parquet('processed_player_stats.parquet')

def add_fixture_datetime(player_stats: pd.DataFrame, matches: pd.DataFrame) -> pd.DataFrame:
    m = matches[['fixture_id','date','timestamp']].copy()
    out = player_stats.merge(m, on='fixture_id', how='left')
    dt_from_str = pd.to_datetime(out['date'], utc=True, errors='coerce')
    dt_from_ts  = pd.to_datetime(out['timestamp'], unit='s', utc=True, errors='coerce')
    out['fixture_dt'] = dt_from_str.fillna(dt_from_ts)
    return out


def add_per90(df: pd.DataFrame, cols=('shots_on','shots_total','passes_key','goals','assists')) -> pd.DataFrame:
    df = df.copy()
    # Ensure minutes is numeric and non-null for math
    df['minutes'] = pd.to_numeric(df['minutes'], errors='coerce').fillna(0)

    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)
            df[f'{c}_per90'] = np.where(df['minutes'] > 0, (df[c] / df['minutes']) * 90.0, 0.0)

    # Availability proxy
    df['minutes_share'] = np.where(df['minutes'] > 0, df['minutes'] / 90.0, 0.0)
    return df

def add_player_ema(df: pd.DataFrame, span=5) -> pd.DataFrame:
    df = df.sort_values(['player_id','fixture_dt']).copy()

    base_cols = []
    if 'games_rating' in df.columns:             # rename to 'rating' for convenience
        df['rating'] = pd.to_numeric(df['games_rating'], errors='coerce')
        base_cols.append('rating')

    # Include the per-90 inputs just created if present
    for c in ['shots_on_per90','shots_total_per90','passes_key_per90','minutes_share']:
        if c in df.columns:
            base_cols.append(c)

    # Shift to use only past information
    for c in base_cols:
        df[f'{c}_shifted'] = df.groupby('player_id', group_keys=False)[c].shift(1)

    # EMA per player (span controls decay speed)
    for c in base_cols:
        ema_col = f'{c}_ema5'
        df[ema_col] = (
            df
            .groupby('player_id', group_keys=False)[f'{c}_shifted']
            .apply(lambda s: s.ewm(span=span, adjust=False).mean())
        )

    # Clean up temporary shifted columns
    df.drop(columns=[f'{c}_shifted' for c in base_cols], inplace=True, errors='ignore')
    return df

def build_player_form_features(player_stats: pd.DataFrame, matches: pd.DataFrame,
                               keep_lead_cols=True, drop_old_features=True) -> pd.DataFrame:
    df = add_fixture_datetime(player_stats, matches)
    df = add_per90(df, cols=('shots_on','shots_total','passes_key','goals','assists'))

    df = add_player_ema(df, span=5)

    # Lean set to keep (customize as needed)
    keep = [
        'fixture_id','player_id','team_id','fixture_dt',
        'rating_ema5','shots_on_per90_ema5','shots_total_per90_ema5',
        'passes_key_per90_ema5','minutes_share_ema5'
    ]
    # Preserve label columns if provided upstream
    if keep_lead_cols:
        keep += [c for c in ('games_rating','minutes') if c in df.columns]

    out = df[[c for c in keep if c in df.columns]].copy()
    return out


In [5]:
# Enrich and process
players_stats = add_fixture_datetime(players_stats, matches)  # adds fixture_dt [3]
processed_df  = process_player_stats(players_stats, drop_old_features=True)
processed_df.to_parquet(f'{BASE_DIR}/processed_player_stats.parquet')

In [6]:
processed_df.describe().round()

,fixture_id,player_id,team_id,rating_ema5,shots_on_per90_ema5,shots_total_per90_ema5,passes_key_per90_ema5,minutes_share_ema5
count,14643.0,14643.0,14643.0,14643.0,14643.0,14643.0,14643.0,14643.0
mean,592480.0,20911.0,47.0,6.0,0.0,1.0,1.0,1.0
std,273.0,40078.0,9.0,2.0,1.0,2.0,1.0,0.0
min,592141.0,17.0,33.0,0.0,0.0,0.0,0.0,0.0
25%,592242.0,1464.0,40.0,6.0,0.0,0.0,0.0,0.0
50%,592337.0,18774.0,46.0,7.0,0.0,0.0,0.0,1.0
75%,592784.0,19128.0,51.0,7.0,0.0,1.0,1.0,1.0
max,592875.0,315237.0,66.0,10.0,90.0,90.0,60.0,1.0


In [7]:
# Build a QA frame that contains fixture_id for mapping checks
# Assumes: processed_df has ['player_id','team_id','fixture_date', *_ema5], and
# players_stats has ['player_id','team_id','fixture_date','fixture_id'] after add_fixture_datetime.

qa = processed_df.merge(
    players_stats[['fixture_id','player_id','team_id','fixture_dt']],
    on=['fixture_id','player_id','team_id','fixture_dt'],
    how='left',
    validate='1:1',      # enforce one-to-one [1][6]
    indicator=True
)
print(qa['_merge'].value_counts(dropna=False))  # expect all 'both' [2]

_merge
both          14643
left_only         0
right_only        0
Name: count, dtype: int64


In [8]:
# 1) Descriptives
print(processed_df.describe().T)  # quick scale/sanity profile [7]

# 2) Missingness
nulls = processed_df.isna().sum().sort_values(ascending=False)  # per-column NA counts [8]
print(nulls.head(10))

# 3) Duplicates on (fixture_id, player_id)
dupes = processed_df.duplicated(subset=['fixture_id','player_id']).sum()  # should be 0 [9]
print("Duplicate (fixture_id, player_id):", dupes)

# 4) Range rules
if 'rating_ema5' in processed_df.columns:
    bad = ~processed_df['rating_ema5'].between(0, 10, inclusive='both')  # typical 0–10 [10]
    print("rating_ema5 out of [0,10]:", int(bad.sum()))
if 'minutes_share_ema5' in processed_df.columns:
    bad = ~processed_df['minutes_share_ema5'].between(0, 1.5, inclusive='both')  # allow some ET [10]
    print("minutes_share_ema5 out of [0,1.5]:", int(bad.sum()))

# 5) IQR outliers for EMA columns
ema_cols = [c for c in processed_df.columns if c.endswith('_ema5')]
for c in ema_cols:
    q1, q3 = processed_df[c].quantile(0.25), processed_df[c].quantile(0.75)  # quartiles [11]
    iqr = q3 - q1
    lo, hi = q1 - 1.5*iqr, q3 + 1.5*iqr
    mask = (processed_df[c] < lo) | (processed_df[c] > hi)
    print(f"{c} IQR outliers:", int(mask.sum()))

# 6) Chronology / leakage: first row per player should have EMA==0 after shift
first_rows = processed_df.sort_values(['player_id','fixture_dt']).groupby('player_id').head(1)
print({c: int(first_rows[c].notna().sum()) for c in ema_cols})  # expect 0 non-NaN before fill [4][5]


                          count           mean           std       min  \
fixture_id              14643.0  592479.640443    272.864097  592141.0   
player_id               14643.0   20911.264905  40078.446851      17.0   
team_id                 14643.0       47.38824      9.322332      33.0   
rating_ema5             14643.0       5.989695      2.283026       0.0   
shots_on_per90_ema5     14643.0       0.325564       1.26251       0.0   
shots_total_per90_ema5  14643.0        0.64833      1.539944       0.0   
passes_key_per90_ema5   14643.0       0.632532      1.249189       0.0   
minutes_share_ema5      14643.0       0.550129      0.382664       0.0   

                             25%       50%       75%        max  
fixture_id              592242.0  592337.0  592784.0   592875.0  
player_id                 1464.0   18774.0   19128.0   315237.0  
team_id                     40.0      46.0      51.0       66.0  
rating_ema5                  6.5  6.760824  7.015372        9.7  
sho